<a href="https://colab.research.google.com/github/gorpo/Bases-python/blob/master/Face_Recognition___Detector_de_Mascara_Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Recognition | Detector de Mascara Covid-19
Considerações: Este script com a camera irá rodar somente no navegador Google Chrome, execute as celulas em ordem e em caso de travamentos pare a maquina por total. Este script aceita sua WebCam Built-in ou uma WebCam tradicional. Caso queira usar seu celular como WebCam aconselho uso do [IvCam para Android](https://www.e2esoft.com/ivcam/). Baixe o cliente em seu celular e o servidor em seu computador depois rode este script.
                                                     @GorpoOrko | 05/2020 | Manicomio Python Project
                                                     ![demonstração](https://github.com/gorpo/Face-Recognition-Detector-de-Mascara-Python-Covid-19/raw/master/exemplos/Screenshot_1.jpg)

# Downloads
Execute a celula de Download e aguarde o download até o final, uma mensagem será exibida informando que todos seus arquivos foram baixados e descompactados com sucesso, estes arquivos incluem um dataset de imagens positivas e negativas com mais de 500 imagens em cada categoria, fique a vontade para inserir mais imagens na pasta /dataset caso ache necessário. Após a conclusão dos dowloads os arquivos seráo automaticamente descompactados, aguarde a mensagem de confirmação:  "Todos arquivos baixados e descompactados com exito inicie o trainer!"
Após esta mensagem inicie o Trainer!

In [0]:
#@title Inicie os Downloads.
from google.colab import output
#baixa o dataset
!wget https://archive.org/download/dataset_202005/dataset.zip
output.clear()
#baixa os arquivos de deteccao
!wget https://archive.org/download/dataset_202005/detectores.zip
output.clear()
#baixa as imagens de exemplo para testes
!wget https://archive.org/download/dataset_202005/exemplos.zip
output.clear()
#dizipando os arquivos baixados
!unzip dataset.zip
output.clear()
!unzip detectores.zip
output.clear()
!unzip exemplos.zip
output.clear()
!rm dataset.zip
!rm detectores.zip
!rm exemplos.zip
#mensagem avisando que o processo acabou
output.clear()
!echo Todos arquivos baixados e descompactados com exito inicie o trainer!

# Trainer
O arquivo trainer irá treinar um 'modelo' de predicção(acertividade) com base nas imagens contidas na pasta dataset previamente baixada, quanto mais imagens 'positivas'(com mascara) e quanto mais imagens 'negativas'(sem mascara) melhor será o resultado, este dataset de imagens conta com mais de 500 imagens em cada categoria. Agora basta executar o Trainer mas antes selecione a quantidade de vezes que ele irá repetir os treinos, quanto maior melhor será a predicção(acertividade). No final do processo será exibido um grafico mostrando a qualidade do seu Trainer, caso queira setar um valor maior de treinos altere o código desta celula.

Selecione a quantidade de treinos e inicie o Trainer

In [0]:
quantidade_treinos = 10 #@param {type:"slider", min:10, max:100, step:1}
from google.colab import output
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


# inicialize a taxa de aprendizado inicial, número de épocas para treinamento e tamanho do lote
inicia_apredizado = 1e-4
#quantidade_treinos = 10
tamanho_lote = 32 #batch_size
# pegue a lista de imagens em nosso diretório de conjunto de dados e inicialize a lista de dados (ou seja, imagens) e imagens de classe
print("[INFO] carregando imagens...")
caminho_imagens = list(paths.list_images("dataset"))
dados = []
labels = []

# loop sobre os caminhos da imagem
for imagePath in caminho_imagens:
	# extrai o rótulo da classe do nome do arquivo
	label = imagePath.split(os.path.sep)[-2]
	# carrega a imagem de entrada (224x224) e pré-processa
	imagem = load_img(imagePath, target_size=(224, 224))
	imagem = img_to_array(imagem)
	imagem = preprocess_input(imagem)
	# atualiza as listas de dados e etiquetas, respectivamente
	dados.append(imagem)
	labels.append(label)

# converte os dados e rótulos em matrizes NumPy
dados = np.array(dados, dtype="float32")
labels = np.array(labels)

# executar codificação one-hot nas etiquetas
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# particione os dados em divisões de treinamento e teste usando 75% dos dados para treinamento e os 25% restantes para teste
(trainX, testX, trainY, testY) = train_test_split(dados, labels, test_size=0.20, stratify=labels, random_state=42)

# construir o gerador de imagens de treinamento para aumento de dados
gerador = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

# carregar a rede MobileNetV2, garantindo que os conjuntos de camadas FC principais sejam deixados de lado
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construa a cabeça do modelo que será colocado em cima do modelo base
modelo_cabeca = baseModel.output
modelo_cabeca = AveragePooling2D(pool_size=(7, 7))(modelo_cabeca)
modelo_cabeca = Flatten(name="flatten")(modelo_cabeca)
modelo_cabeca = Dense(128, activation="relu")(modelo_cabeca)
modelo_cabeca = Dropout(0.5)(modelo_cabeca)
modelo_cabeca = Dense(2, activation="softmax")(modelo_cabeca)

# coloque o modelo -FACE RECOGNITION- principal sobre o modelo base (este se tornará o modelo real que iremos treinar)
modelo = Model(inputs=baseModel.input, outputs=modelo_cabeca)

# percorre todas as camadas no modelo base e as congela para que elas * não * sejam atualizadas durante o primeiro processo de treinamento
for layer in baseModel.layers:
	layer.trainable = False

# compile nosso modelo
print("[INFO] compilando o modelo...")
otimizador = Adam(lr=inicia_apredizado, decay=inicia_apredizado / quantidade_treinos)
modelo.compile(loss="binary_crossentropy", optimizer=otimizador, metrics=["accuracy"])

# treinar a face
print("[INFO] iniciando treinamento das faces...")
cabeca = modelo.fit(gerador.flow(trainX, trainY, batch_size=tamanho_lote), steps_per_epoch=len(trainX) // tamanho_lote, validation_data=(testX, testY), validation_steps=len(testX) // tamanho_lote, epochs=quantidade_treinos)
output.clear()
# faça previsões(predicções) no conjunto de testes
print("[INFO] avaliação de rede neural...")
predIdxs = modelo.predict(testX, batch_size=tamanho_lote)
# para cada imagem no conjunto de testes, precisamos encontrar o índice do rótulo com a maior probabilidade prevista correspondente
predIdxs = np.argmax(predIdxs, axis=1)
# mostrar um relatório de classificação
print(classification_report(testY.argmax(axis=1), predIdxs,	target_names=lb.classes_))
# salve o modelo
print("[INFO] salvando o modelo detector de mascara...")
modelo.save("detectores/mask_detector.model", save_format="h5")
# traçar a perda e a precisão do treinamento
N = quantidade_treinos
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), cabeca.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), cabeca.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), cabeca.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), cabeca.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("detectores/plot.png")

# Analise de Imagens
A células abaixo fazem a inserção das imagens e a análise em imagens estaticas verificando a acertividade em reconhecer o uso ou não da máscara em pessoas. Este script precisa da inserção de uma imagem via link ou upload, ou seja, basta adicionar a url de uma imagem que tenha o final .jpg ou .png conforme 
[exemplo](https://i.imgur.com/heVp6Jj.jpg) ou fazer o upload da sua imagem na proxima célula, lembre que só pode ser usada uma célula por vez. Após inserido link da imagem ou feito upload de uma imagem do seu computador basta iniciar a célula ""Analise de Imagens"" para iniciar o reconhecimento facial.

 **Upload da imagem via Link:**
<br>Insira um link válido com final .jpg ou .png, em caso de erros  use o sistema de upload de imagens na celula abaixo

In [0]:
#SISTEMA DE ANALISE DE IMAGENS COM MASCARA COVID19
#download da imagem a ser analisada:

url = "https://github.com/gorpo/Face-Recognition-Detector-de-Mascara-Python-Covid-19/blob/master/exemplos/1.png" #@param {type:"string"}

print(url)
import requests
import time

#url = input()
r = requests.get(url)
with open('exemplos/file.png', 'wb') as f:
    f.write(r.content)
print('Download da imagem concluido com sucesso, inicie o processo de análise...')		




 **Upload de imagens do seu computador:**

In [0]:
#@markdown Rode esta célula e no botão 'Escolher arquivos', selecione a imagem que deseja fazer o upload.

#faça upload da sua imagem do computador

from google.colab import files
import shutil
uploaded = files.upload()
for fn in uploaded.keys():
  print('[INFO] Usuario fez upload do arquivo "{name}" com tamanho {length} bytes\n[INFO] Imagem salva na pasta exemplos/file.png\n[INFO] Upload concluido com sucesso rode a Analise de Imagens.'.format(name=fn, length=len(uploaded[fn])))
shutil.move(fn, 'exemplos/file.png')

 **Análise de Imagens**

In [0]:
#@markdown Após ter feito upload da sua imagem com sucesso basta rodar esta célula para realizar o sistema de reconhecimento de imagens de pessoas com ou sem máscara!


#inicio do reconhecimento facial e de mascaras:
from google.colab.patches import cv2_imshow
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os


try:
	# carregar nosso modelo de detector de rosto serializado a partir do disco
	print("[INFO] carregando modelo detector de faces...")
	prototxtPath = os.path.sep.join(["detectores/deploy.prototxt"])
	weightsPath = os.path.sep.join(	["detectores/res10_300x300_ssd_iter_140000.caffemodel"])
	net = cv2.dnn.readNet(prototxtPath, weightsPath)
	# carrega o detector de mascara facialload the face mask detector model from disk
	print("[INFO] carregando detector de mascara facial...")
	modelo = load_model("detectores/mask_detector.model")

	# carregue a imagem, clone-a e pegue as dimensões espaciais da imagem
	imagem = cv2.imread('exemplos/file.png')
	origem = imagem.copy()
	(h, w) = imagem.shape[:2]

	# construir um blob a partir da imagem
	blob = cv2.dnn.blobFromImage(imagem, 1.0, (300, 300), (104.0, 177.0, 123.0))
	# passe o blob pela rede e obtenha as detecções de rosto
	print("[INFO] computando deteccoes faciais...")
	net.setInput(blob)
	deteccoes = net.forward()


	# loop sobre as detecçoes
	for i in range(0, deteccoes.shape[2]):
		# extrair a confiança (ou seja, probabilidade) associada à detecção
		confianca = deteccoes[0, 0, i, 2]
		# filtrar detecções fracas, garantindo que a confiança seja maior que a confiança mínima
		if confianca > 0.5:
			# calcular as coordenadas (x, y) da caixa delimitadora para o objeto
			box = deteccoes[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			# verifique se as caixas delimitadoras estão dentro das dimensões do quadro
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))
			# extrai o ROI da face, converte-o de pedido de canal BGR para RGB, redimensione-o para 224x224 e pré-processe
			face = imagem[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)
			face = np.expand_dims(face, axis=0)
			# passe o rosto pelo modelo para determinar se o rosto tem uma máscara ou não
			(mask, withoutMask) = modelo.predict(face)[0]
			# determine a label e a cor da classe que usaremos para desenhar a caixa delimitadora e o texto
			label = "com mascara" if mask > withoutMask else "sem mascara"
			color = (0, 255, 0) if label == "com mascara" else (0, 0, 255)
			# inclua a probabilidade na label
			label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
			# exibir a label e o retângulo da caixa delimitadora no quadro de saída
			cv2.putText(imagem, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
			cv2.rectangle(imagem, (startX, startY), (endX, endY), color, 2)

	# Mostra a imagem final
	cv2_imshow(imagem)
	cv2.waitKey(0)
except:
	print('[ATENÇÃO] Ocorreu um erro ao processar sua imagem.\n[MOTIVO] Mesmo ela tendo o final .jpg/.png ela não tinha as propriedades necessarias, devido ter vindo de uma fonte não confiavel.\n[CORREÇÃO] Aconselho upload de suas imagens para analise no https://i.imgur.com/, insira o link da imagem upada com final .jpg ou .png') 

# Analise com WebCam
O sistema de análise de dados com a WebCam terá sua precisão com variação na qualidade da camera e da luz ambiente, este script roda tanto com uma camera local(built-in) como uma camera de celular, para usar seu celular como camera via PC aconselho o uso do [IvCam para Android](https://www.e2esoft.com/ivcam/). Baixe o cliente em seu celular e o servidor em seu computador depois rode este script. Porém caso queira usar direto seu telefone como camera, este script roda tranquilamente no navegador de celulares Google Chrome!

In [0]:
#@title Inicie esta célula para fazer analise de dados via webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import imutils
import time
import os



def VideoCapture():
#inicio do codigo padrão para usar a camera no google colab conforme documentação
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);
      video = document.createElement('video');
      video.setAttribute('playsinline', '');
      div.appendChild(video);
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;
      await video.play();
      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }
    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }''')
  display(js)
def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)
def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x
VideoCapture()
eval_js('create()')  
#final do codigo padrao para uso de camera no google colab, para chamar a camera no resto do script usaremos:
#byte = eval_js('capture()')
#imagem = byte2image(byte)
#onde imagem ira retornar nossa imagem ja convertida para bytes
#------------------------------------------------------------------------------

try:
  def deteccao(frame, faceNet, maskNet):
    # pegue as dimensões do quadro e construa um blob a partir dele
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))
    # passe o blob pela rede e obtenha as detecções de rosto
    faceNet.setInput(blob)
    detections = faceNet.forward()
    # inicialize nossa lista de rostos, seus locais correspondentes e a lista de previsões da nossa rede de máscaras faciais
    faces = []
    locs = []
    preds = []
    # loop nas detecções
    for i in range(0, detections.shape[2]):
      # extrair a confiança (ou seja, probabilidade) associada à detecção
      confidence = detections[0, 0, i, 2]
      # filtrar detecções fracas, garantindo que a confiança seja maior que a confiança mínima
      if confidence > 0.5:
        # calcular as coordenadas (x, y) da caixa delimitadora para o objeto
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        # verifique se as caixas delimitadoras estão dentro das dimensões do quadro
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        # extrai o ROI da face, converte-o de pedido de canal BGR para RGB, redimensione-o para 224x224 e pré-processe-o
        face = frame[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        # adicione as caixas de rosto e delimitadoras às respectivas listas
        faces.append(face)
        locs.append((startX, startY, endX, endY))
    # apenas faça previsões se pelo menos uma face for detectada
    if len(faces) > 0:
      # para uma inferência mais rápida, faremos previsões de lote em * todos * rostos ao mesmo tempo, em vez de previsões um por um no loop `for` acima
      preds = maskNet.predict(faces)
    # retorna duas tuplas dos locais de face e seus locais correspondentes
    return (locs, preds)


  # carregar nosso modelo de detector de rosto serializado a partir do disco
  print("[INFO] carregando modelo de detector de rosto ...")
  prototxtPath = os.path.sep.join(["detectores/deploy.prototxt"])
  weightsPath = os.path.sep.join(	["detectores/res10_300x300_ssd_iter_140000.caffemodel"])
  faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
  # carregar o modelo do detector de máscara facial a partir do disco
  print("[INFO] carregando modelo de detector de máscara facial ...")
  maskNet = load_model("detectores/mask_detector.model")
  # inicialize o fluxo de vídeo e permita que o sensor da câmera capte imagens
  print("[INFO] iniciando video...")
  print("[INFO] iniciando reconhecimento facial...")
  #mude o valor da camera para 0 caso a camera seja built-in ou  troque pelo ip de sua camera-->> 'http://192.168.0.4:4747/mjpegfeed'

  # loop sobre o vídeo
  while True:

    # pegue o quadro do fluxo de vídeo encadeado e redimensione-o para ter uma largura máxima de 400 pixels
    #linha 130 e 131 sao o retorno da camera(imagem em video)
    byte = eval_js('capture()')
    imagem = byte2image(byte)

    #imagem = video.read()
    imagem = imutils.resize(imagem, width=400)
    # detectar rostos no quadro e determinar se eles estão usando uma máscara facial ou não

    (locs, preds) = deteccao(imagem, faceNet, maskNet)
    # circula sobre os locais de face detectados e seus locais correspondentes
    for (box, pred) in zip(locs, preds):
      # descompacte a caixa delimitadora e as previsões
      (startX, startY, endX, endY) = box
      (mask, withoutMask) = pred
      # determine o rótulo e a cor da classe que usaremos para desenhar a caixa delimitadora e o texto
      label = "com mascara" if mask > withoutMask else "sem mascara"
      color = (0, 255, 0) if label == "com mascara" else (0, 0, 255)
      # inclui a probabilidade em uma label
      label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
      # exibe os textos da label no frame
      cv2.putText(imagem, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
      cv2.rectangle(imagem, (startX, startY), (endX, endY), color, 2)

    # mostra o video
    eval_js('showimg("{}")'.format(image2byte(imagem)))
except:
  print('Algum erro ocorreu no reconhecimento tente novamente!')

![@GorpoOrko 2020 | tcxsproject.com.br](https://raw.githubusercontent.com/gorpo/Manicomio-Boot-Theme/master/manicomio/boot.png)